In [1]:
from mssubsys_server.ta_formula.indicators import *
from test_data import test_data

In [1]:
import logging

from mdsubsys_client.client import *
from mssubsys_server.ta_formula import data_context, get_strategy_class
from mssubsys_server.ta_formula.datasource import (AioDataBackend, DataBackend,
                                                   close_all_backends,
                                                   add_backend)

logging.basicConfig(level='DEBUG')

class MdsubsysDataBackend(DataBackend):
    prefix = 'mdsubsys_server'

    def __init__(self, name, config) -> None:
        super().__init__(name, config)
        self.client = init_client(config['ip'], config['port'])
        self.bars = None

    def prepare(self, symbols, intervals):
        # 下载的K线或者Tick数据可以做缓存，避免多次调用重复下载
        self.bars = self.client.get_bars(symbols=symbols, intervals=intervals)

    def get_data(self, symbol, interval):
        return self.bars[symbol][interval]

    def release(self, symbols, intervals):
        pass

    def close(self):
        self.client.close()

# class MdsubsysDataBackend(AioDataBackend):

#     def __init__(self, name, config) -> None:
#         super().__init__(name, config)
#         self.client = init_client(config['ip'], config['port'], on_bars_func=on_bars, is_asyncio=True)

#     async def prepare(self, symbols, intervals):
#         await self.client.subscribe_bars(symbols=symbols)
#         while True:
#             _last_ticking_bars = _init_ticking_bars(symbols, interval)
#             if _last_ticking_bars:
#                 break
#             await asyncio.sleep(0.5)

#     async def release(self, symbols, intervals):
#         return super().release()

#     async def close(self):
#         pass

request = {
    "datasources": [["mdsubsys_server.ctp.ag2412"], ["mdsubsys_server.ctp.al2408"]],
    "version":"v1_14F",
    "strategy_name":"shortswingkiller",
    "return_fields": ["open", "close", "CLOSE[-1]"],
    "params": {}
}
data_backends = []
data_backends.append(add_backend("mdsubsys_server.ctp", {"ip": "192.168.2.9", "port": 50051}))
strategy_cls = get_strategy_class('mssubsys_server.strategies', request['strategy_name'], request['version'])
strategy = strategy_cls(request['params'], request['return_fields'])

%load_ext line_profiler

with data_context(request['datasources'], strategy_cls._intervals) as datas_itr:
    for datas_lst in datas_itr:
        # %timeit strategy.calculate(datas_lst)
        # print(strategy.calculate(datas_lst))
        # %prun strategy.calculate(datas_lst)
        # %lprun -f strategy.calculate strategy.calculate(datas_lst)

        %timeit strategy.output(datas_lst)
        print(strategy.output(datas_lst))

close_all_backends()

[('mdsubsys_server.ctp', ['al2408', 'ag2412'], ('1m', '5m', '15m'))]
279 µs ± 5.71 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
{'symbol': None, 'interval': '1m', 'open_long_condition': False, 'open_short_condition': False, 'close_long_condition': False, 'close_short_condition': False}


INFO:root:Mdsubsys client quit the grpc channel
INFO:root:数据后台 mdsubsys_server.ctp 关闭


278 µs ± 4.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
{'symbol': None, 'interval': '1m', 'open_long_condition': False, 'open_short_condition': False, 'close_long_condition': False, 'close_short_condition': False}
